<a href="https://colab.research.google.com/github/marinwitherspoon/English-monarchs/blob/main/Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# url of scrape
url = 'https://en.wikipedia.org/wiki/List_of_English_monarchs'

# Get the HTML content using requests
html_content = requests.get(url).text

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

all_king_tables = soup.find_all('table',{'class':"wikitable"})

df_all = pd.DataFrame(columns=['Name', 'Birth', 'Death', 'Claim'])

for i in all_king_tables:
  df=pd.read_html(str(i))[0]

  if 'Claim' in df.columns:
    selected_columns = ['Name', 'Birth', 'Death', 'Claim']
  else:
    selected_columns = ['Name', 'Birth', 'Death']
    df['Claim'] = None

  # convert list to dataframe
  df=pd.DataFrame(df)[selected_columns]

  df_all = pd.concat([df_all, df], axis=0, ignore_index=True)

#  clean data  ##########################

# Delete rows with invalid data
df_all.drop([18, 25, 48], inplace=True)
#reset indexing
df_all.reset_index(drop=True, inplace=True)

# - clean up name column -------------------
#seperate the name form the rest of the string
df_all['Name'] = df_all['Name'].str.replace(r'\[.*?\]', '')

#cleaning name column to include only names
df_all[['Name', 'desc']] = df_all['Name'].str.split(r'\s\d|\[\d+\]', 1, expand=True)
df_all['Name'] = df_all['Name'].apply(lambda row: ' '.join(sorted(set(row.split()), key=row.index)))

df_all['Name'][0] = 'Alfred the Great'

#exstract dates from description
df_all['dates'] = df_all['desc'].str.extract(r'(\s*\d{3,4}\s*(?:–\s*\d{0,2}\s*\w*\s*\d{3,4})?)')
df_all['dates'][29] = '1307 – 1327'
df_all['dates'][3] = '927 – 939'
for i in df_all['dates']:
  bre = i.split(' - ')
  df_all['start'] = bre[0]
#remove days and months
df_all['dates'] = df_all['dates'].str.replace(r'\s+\d{0,2}\s+\w*\s+', ' ')

for index, row in df_all.iterrows():
  date_range = row['dates'].strip().split('–')

  df_all.at[index, 'start'] = int(date_range[0].strip())
  df_all.at[index, 'end'] = int(date_range[1].strip())

# - clean up Birth column -------------------
df_all['Birth'] = df_all['Birth'].str.extract(r'(\d{3,4})').astype(int)

# - clean up Death column -------------------
df_all['Death'] = df_all['Death'].str.extract(r'(\d{3,4})').astype(int)

# - clean up Claim column -------------------
df_all['Claim'] = df_all['Claim'].str.replace(r'/.*?of', 'of')
#exstract relationship
df_all['ClaimRelation'] = df_all['Claim'].str.extract(r'(\w*(?:-\w+)*(?:Son|Daughter))\s+',flags=re.IGNORECASE)
# exstract and clean up names
df_all['Claim'] = df_all['Claim'].str.extract(r'(?:Son|Daughter)\s+of\s+(\w+\s*(?:[IV]+|of\s+\w+|the\s+\w+)*)',flags=re.IGNORECASE)
df_all['Claim'] = df_all['Claim'].str.replace(r'(?<=[^ ])(?<![IV])([A-Z])(.*?)$','')

#manually correct data
df_all['Claim'][10] = 'NaN'
df_all['Claim'][45] = 'spouse'

#sort table by coorination date
df_all = df_all.sort_values(by='start').reset_index(drop=True)


<ipython-input-18-286531b4a09e>:42: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all['Name'] = df_all['Name'].str.replace(r'\[.*?\]', '')
<ipython-input-18-286531b4a09e>:45: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_all[['Name', 'desc']] = df_all['Name'].str.split(r'\s\d|\[\d+\]', 1, expand=True)
<ipython-input-18-286531b4a09e>:58: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all['dates'] = df_all['dates'].str.replace(r'\s+\d{0,2}\s+\w*\s+', ' ')
<ipython-input-18-286531b4a09e>:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all['Claim'] = df_all['Claim'].str.replace(r'/.*?of', 'of')
<ipython-input-18-286531b4a09e>:78: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all['Claim'] =

In [ ]:
!pip3 install ColabTurtlePlus
!pip3 install cairosvg

In [3]:
import ColabTurtlePlus.Turtle as T

def sqline(length, depth, color):
  T.color(color)
  T.begin_fill()

  T.left(90)
  T.forward(depth/2)

  T.right(90)
  T.forward(length)

  T.right(90)
  T.forward(depth)

  T.right(90)
  T.forward(length)

  T.right(90)
  T.forward(depth/2)

  T.right(90)
  T.end_fill()
  T.forward(length)

Put clearscreen() as the first line in a cell (after the import command) to re-run turtle commands in the cell


In [19]:
import ColabTurtlePlus.Turtle as T
import cairosvg

date_range = max(df_all['Death'])-min(df_all['Birth'])+20

T.reset()
# set screen size, starting pen location and pen size
screen = T.Screen()
y = 600
screen.setup(date_range, y)
screen.setworldcoordinates(0, 0, date_range+100, y+10)
T.penup()
T.speed(15)

#### make time line -------------------------------
T.goto(10, 10)
T.left(90)

for i in range(8,9 + round(date_range/100)):
  T.pensize(2)
  T.color('dark grey')
  T.goto(i*100-int(df_all['Birth'][0]), 10)
  T.write(i*100, font = ('Arial', 8, 'normal'))
  T.pendown()
  T.forward(y)
  T.penup()
  for j in range(1,4):
    T.color('grey')
    T.goto(i*100+25*j-int(df_all['Birth'][0]), 10)
    if j == 2:
      T.pensize(1)
      T.write(i*100+25*j, font = ('Arial', 8, 'normal'))
    else:
      T.pensize(.5)
    T.pendown()
    T.forward(y)
    T.penup()
T.right(90)
# -----------------------------------
T.goto(10, y-10)
T.pensize(.1)

iSkip = []

for i in range(len(df_all)):
  #skip if its their second reign
  if i in iSkip:
    continue

  T.goto(int(df_all['Birth'][i]-df_all['Birth'][0]+10), y-10*i)
  T.pendown()

  # find and draw lineage
  if any(df_all['Name'].str.contains(str(df_all['Claim'][i]), case=False, na=False)):
    T.pensize(1)
    lineage = df_all.index[df_all['Name'].str.contains(df_all['Claim'][i], case=False, na=False)].tolist()[0]
    T.goto(int(df_all['Birth'][lineage]-df_all['Birth'][0]+10), y-10*i)
    T.goto(int(df_all['Birth'][lineage]-df_all['Birth'][0]+10), y-10*lineage)
    T.penup()
    T.goto(int(df_all['Birth'][i]-df_all['Birth'][0]+10), y-10*i)
    T.pensize(.1)
    T.pendown()

  #life before crown
  sqline(int(df_all['start'][i]) - int(df_all['Birth'][i]), 8, 'black')
  #life during crown
  sqline(df_all['end'][i] - df_all['start'][i], 8, 'orange')
  #if they had a life after crown

  N = df_all['Name'][i]

  if df_all['Death'][i] - df_all['end'][i] != 0:
    sqline(df_all['Death'][i] - df_all['end'][i], 8, 'black')
    #check if they had a second reign
    if "(1st reign)" in df_all['Name'][i]:
      N = df_all['Name'][i].strip("(1st reign)")
      ind = df_all.index[df_all['Name'] == '(2nd reign) ' + N].tolist()[0]
      sqline(df_all['end'][ind] - df_all['start'][ind], 8, 'orange')
      #check if they had a life after their second reign
      if df_all['Death'][ind] - df_all['end'][ind] != 0:
        sqline(df_all['Death'][i] - df_all['end'][i], 8, 'black')
      iSkip.append(ind)
    else:
      N = df_all['Name'][i]

  T.penup()
  T.forward(10)
  T.color("black")
  T.write(N, font = ('Arial', 8, 'normal'))

T.goto(-100,0)

T.saveSVG(turtle=False)

cairosvg.svg2pdf(url='SVGimage.svg', write_to='drawing.pdf')
# add connection
# change claim to generation count
# add generations?
# add legend

In [ ]:
df_all